In [ ]:
import numpy as np
import matplotlib
matplotlib.use('Agg')   
import seaborn as sns
import pandas as pd
import os
%matplotlib inline

plt=sns.plt
use=10000
th=0.01

f = open('<path to metafiles>/filtered_conditions.txt')
conditions = f.read().split('\n')
f.close()
conditions = [conditions[x] for x in range(np.shape(conditions)[0]-1)]

f = open('<path to metafiles>/filtered_vocabulary.txt')
vocabulary = f.read().split('\n')
f.close()
vocabulary = [vocabulary[x] for x in range(np.shape(vocabulary)[0]-1)]

'''Parameter to set '''
s = 0.01
mm="logistic"
s = 0.1
mm="cnmf"
#mm="nmf"
#mm='llda'

if mm.startswith('nmf'):
    s=0
    minIdx=0
    fval = np.inf
    for idx in range(6):
        result = np.load('<result_pickle> for fold idx')
        if result['lossValuefinal'] < fval:
            minIdx = idx
            fval = result['lossValuefinal']
    i = minIdx
    mm="NMF_s%d_i%d" %(s*100,i)   
    if not os.path.exists(mm): os.mkdir(mm)
    print s,i, mm
    result = np.load('<result_pickle> for fold i')
    phenotype_matrix = result['A']
    phenotype_matrix = phenotype_matrix/phenotype_matrix.sum(0)
    print result['A'].shape
    assert result['A'].shape[0]==len(vocabulary), 'NMF vocabulary dimension mismatch'

elif mm.startswith('cnmf'):
    minIdx=1
    fval = np.inf
    for idx in range(1,6):
        result = np.load('<result_pickle> for fold idx')
        if result['lossValuefinal'] < fval:
            minIdx = idx
            fval = result['lossValuefinal']
    i = minIdx
    mm="CNMF_s%d_i%d" %(s*100,i)
    if not os.path.exists(mm): os.mkdir(mm)
    print s,i, mm
    result = np.load('<result_pickle> for fold i')
    phenotype_matrix = result['A']
    phenotype_matrix = phenotype_matrix/phenotype_matrix.sum(0)
    print result['A'].shape
    assert result['A'].shape[0]==len(vocabulary), 'CNMF vocabulary dimension mismatch'
elif mm.startswith('logistic'):
    mm="logistic_s%d" %(s*100) 
    if not os.path.exists(mm): os.mkdir(mm)
    print s,i, mm
    result = np.load('<result_pickle> for s')
    phenotype_matrix = result['A']
    phenotype_matrix = np.clip(phenotype_matrix,0,None)
    phenotype_matrix = phenotype_matrix/phenotype_matrix.sum(0)
    print result['A'].shape
    assert result['A'].shape[0]==len(vocabulary), 'Logistic vocabulary dimension mismatch'

elif (mm=="llda"):
    print mm
    if not os.path.exists(mm): os.mkdir(mm)
    data = pd.read_csv('<path to llda result keys (*.keys)>'
                        ,sep='\t',header=None,names=['id','topic','nterms','top-20'])
    llda_hasbias = 1
    if llda_hasbias:
        llda_conditions = conditions + ['bias']
    slist=[]
    for i in range(data.shape[0]):
        s = data['top-20'][i].split(' ')
        s = [s[x] for x in range(min(len(s)-1,100000))]
        slist.append(s)
    data['top-20'] = slist
    llda_pheno_data = pd.DataFrame()
    for i in range(data.shape[0]):
        llda_pheno_data[llda_conditions[data['topic'][i]]] = data['top-20'][i]
    llda_pheno_data = llda_pheno_data.drop('bias',axis=1)
    

R = len(conditions)

phenotypes={}
phenotypes_scores={}
anchor_features={}
anchor_scores={}
all_pheno_ids = []
if mm=='llda':
    for r in range(R):
        phenotypes[conditions[r]] = llda_pheno_data[conditions[r]]
        phenotypes_scores[conditions[r]]=np.ones(15)
else:
    for r in range(R):
        pheno_r = np.where(phenotype_matrix[:,r]>1e-10)[0]
        features_r = [vocabulary[x] for x in pheno_r]
        # sort features in descending order
        pheno_r_scores = phenotype_matrix[pheno_r,r]
        pheno_r_scores = 0.5+0.5*(pheno_r_scores-min(pheno_r_scores))/(max(pheno_r_scores)-min(pheno_r_scores))
        sorted_idx = np.argsort(pheno_r_scores)[::-1]
        phenotypes[conditions[r]] = [features_r[x] for x in sorted_idx]
        phenotypes_scores[conditions[r]] = [pheno_r_scores[x] for x in sorted_idx]
        if r==8:
            print conditions[r],phenotypes_scores[conditions[r]]
        all_pheno_ids.extend(sorted_idx)

In [ ]:
plt=matplotlib.pyplot
sns.set_context("poster", font_scale=1, rc={"lines.linewidth": 1})
col=sns.color_palette("BuGn_r",50)
plt.figure(figsize=(25,30))
sc=1
for r in range(30):
    ax=plt.subplot(5, int(R/5), r + 1)  # plot numbering starts with 1
    ax.set(xticks=[],yticks=[]) # remove axis markings ('ticks')
    ymax=ax.get_ylim()[1]-0.01
    plt.title(str(conditions[r]), color="blue")

    ypos=0.0         
    fs=18
    for i, (word, share) in enumerate(zip(phenotypes[conditions[r]], phenotypes_scores[conditions[r]])):   
        if (i==25):
            plt.text(0.01, ymax-ypos, "+%d" %(len(phenotypes[conditions[r]])-25), fontsize=0.7*fs*sc, verticalalignment='top')
            break
        plt.text(0.01, ymax-ypos, word,fontsize=fs*share*sc,color=col[20*(i/15)],verticalalignment='top')
        ypos=ypos+1.0/27


plt.tight_layout(rect=[0, 0.01, 0.99, 1],pad=0.1)
plt.savefig('phenotype_%s.pdf' %mm)